In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from lxml import etree
from logos_tools import *
from logos_opta import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit, KFold, GridSearchCV, PredefinedSplit
from sklearn.linear_model import LogisticRegression

In [2]:
EVENT_FILE = 'event.txt'
QUALIFIER_FILE = 'qualifier.txt'
ASSOCIATE_FILE = 'associate_use.tsv'
GAME_FILE = 'test.xml'
PLAYER_FILE = 'players.tsv'
USE_PLAYER_FILE = 'use_player'
player_transformer_file = 'player_transformer'
player_model_file = 'player_model'
next_team_transformer_file = 'next_team_transformer'
next_team_model_file = 'next_team_model'
next_x_transformer_file = 'next_x_transformer'
next_x_model_file = 'next_x_model'
next_y_transformer_file = 'next_y_transformer'
next_y_model_file = 'next_y_model'

USE_PLAYERID = '1'

In [3]:
# load dim file
event_df = pd.read_csv(EVENT_FILE, sep='|')
event_s = pd.Series(data=event_df.type.values, index=[str(x) for x in event_df.id])
event_dict = event_s.to_dict()
qualifier_df = pd.read_csv(QUALIFIER_FILE, sep='|')
qualifier_s = pd.Series(data=qualifier_df.type.values, index=[str(x) for x in qualifier_df.id])
qualifier_dict = qualifier_s.to_dict()
associate_df = pd.read_csv(ASSOCIATE_FILE, sep='\t')
associate_dict = {str(associate_df.Type_id[i]):associate_df.qualifier_id[i].split(',') for i in range(len(associate_df))}


In [4]:
def get_player_sample(use_df, start, end):
    use_half = '2' if start > 45*60 else '1'
    # delete last 10 event and stat
    use_df = use_df.iloc[:-10]
    # game period
    use_period = get_period(use_half, start)
    # shoot stat
    shoot_df = pd.DataFrame({'bigChance':[0], 'bigChance_rate':[0.0], 'head':[0], 'head_rate':[0.0], 
                             'inside':[0], 'inside_rate':[0.0], 'onTarget':[0], 'onTarget_rate':[0.0], 
                             'outside':[0], 'outside_rate':[0.0], 'shoot':[0]})
    selected = use_df.loc[use_df.event_type.isin(['Miss', 'Post', 'Attempt Saved', 'Goal'])]
    selected = selected.loc[selected.player_id==USE_PLAYERID]
    if len(selected)>0:
        shoot_df = shoot_stat(selected)
    # pass stat
    pass_df = pd.DataFrame({'pass':[0], 'pass1_rate':[0.0], 
                     'front':[0], 'front_rate':[0.0], 'front1_rate':[0.0],
                     'key':[0], 'key_rate':[0.0],
                     'cross':[0], 'cross_rate':[0.0], 'cross1_rate':[0.0],
                     'through':[0], 'through_rate':[0.0], 'through1_rate':[0.0],
                     'chipped':[0], 'chipped_rate':[0.0], 'chipped1_rate':[0.0],
                     'forward':[0], 'forward_rate':[0.0], 'forward1_rate':[0.0],
                     'back':[0], 'back_rate':[0.0], 'back1_rate':[0.0],
                     'left':[0], 'left_rate':[0.0], 'left1_rate':[0.0],
                     'right':[0], 'right_rate':[0.0], 'right1_rate':[0.0],
                     'short':[0], 'short_rate':[0.0], 'short1_rate':[0.0],
                     'middle':[0], 'middle_rate':[0.0], 'middle1_rate':[0.0],
                     'long':[0], 'long_rate':[0.0], 'long1_rate':[0.0],
                     'positionC':[0], 'positionC_rate':[0.0], 'positionC1_rate':[0.0],
                     'positionL':[0], 'positionL_rate':[0.0], 'positionL1_rate':[0.0],
                     'positionR':[0], 'positionR_rate':[0.0], 'positionR1_rate':[0.0],
                     'positionB':[0], 'positionB_rate':[0.0], 'positionB1_rate':[0.0]
                    })
    selected = use_df.loc[use_df.event_type=='Pass']
    selected = selected.loc[selected.player_id==USE_PLAYERID]
    if (len(selected)>0):
        pass_df = pass_stat(selected)
    # other stat
    selected = use_df.loc[use_df.player_id==USE_PLAYERID]
    other_df = other_stat(selected)
    # get team_id
    team_id_s = pd.Series(selected.team_id.iloc[0], name='team_id')
    # concat
    sample = pd.concat([pass_df, shoot_df, other_df, team_id_s], axis=1, sort=False)
    sample = sample.fillna(0)
    sample['period'] = use_period
    sample['start'] = start
    sample['end'] = end
    return sample

In [15]:
def get_team_sample(use_df, start, end):
    # game period
    use_half = '2' if start > 45*60 else '1'
    use_period = get_period(use_half, start)
    # last 10 event
    last10 = use_df.iloc[-10:]
    last10_list = list(last10.team_id +'-'+ last10.event_type +'-'+ last10.apply(lambda row: get_position_xy(row.x, row.y), axis=1) +'-'+ use_period)
    last_team = last10.team_id.iloc[-1]
    last_event_type = last10.event_type.iloc[-1]
    last_x = safe_convert(last10.x.iloc[-1], float, 0)
    last_y = safe_convert(last10.y.iloc[-1], float, 0)
    last_event = last10_list[-1]
    # delete last 10 event and stat
    use_df = use_df.iloc[:-10]
    # shoot stat
    shoot_df = pd.DataFrame({'bigChance':[0,0], 'bigChance_rate':[0.0,0.0], 'head':[0,0], 'head_rate':[0.0,0.0], 'inside':[0,0], 'inside_rate':[0.0,0.0], 'onTarget':[0,0], 'onTarget_rate':[0.0,0.0], 'outside':[0,0], 'outside_rate':[0.0,0.0], 'shoot':[0,0]}, index=['0','1'])
    shoot_df.index.name = 'team_id'
    selected = use_df.loc[use_df.event_type.isin(['Miss', 'Post', 'Attempt Saved', 'Goal'])]
    if len(selected)>0:
        shoot_df = selected.groupby('team_id').apply(shoot_stat)
        shoot_df = shoot_df.reset_index(level=1, drop=True)
    # pass stat
    pass_df = pd.DataFrame({'pass':[0,0], 'pass1_rate':[0.0,0.0], 
                         'front':[0,0], 'front_rate':[0.0,0.0], 'front1_rate':[0.0,0.0],
                         'key':[0,0], 'key_rate':[0.0,0.0],
                         'cross':[0,0], 'cross_rate':[0.0,0.0], 'cross1_rate':[0.0,0.0],
                         'through':[0,0], 'through_rate':[0.0,0.0], 'through1_rate':[0.0,0.0],
                         'chipped':[0,0], 'chipped_rate':[0.0,0.0], 'chipped1_rate':[0.0,0.0],
                         'forward':[0,0], 'forward_rate':[0.0,0.0], 'forward1_rate':[0.0,0.0],
                         'back':[0,0], 'back_rate':[0.0,0.0], 'back1_rate':[0.0,0.0],
                         'left':[0,0], 'left_rate':[0.0,0.0], 'left1_rate':[0.0,0.0],
                         'right':[0,0], 'right_rate':[0.0,0.0], 'right1_rate':[0.0,0.0],
                         'short':[0,0], 'short_rate':[0.0,0.0], 'short1_rate':[0.0,0.0],
                         'middle':[0,0], 'middle_rate':[0.0,0.0], 'middle1_rate':[0.0,0.0],
                         'long':[0,0], 'long_rate':[0.0,0.0], 'long1_rate':[0.0,0.0],
                         'positionC':[0,0], 'positionC_rate':[0.0,0.0], 'positionC1_rate':[0.0,0.0],
                         'positionL':[0,0], 'positionL_rate':[0.0,0.0], 'positionL1_rate':[0.0,0.0],
                         'positionR':[0,0], 'positionR_rate':[0.0,0.0], 'positionR1_rate':[0.0,0.0],
                         'positionB':[0,0], 'positionB_rate':[0.0,0.0], 'positionB1_rate':[0.0,0.0]
                        }, index=['0','1'])
    pass_df.index.name = 'team_id'
    selected = use_df.loc[use_df.event_type=='Pass']
    if (len(selected)>0):
        pass_df = selected.groupby('team_id').apply(pass_stat)
        pass_df = pass_df.reset_index(level=1, drop=True)
    # other stat
    other_df = use_df.groupby('team_id').apply(other_stat)
    other_df = other_df.reset_index(level=1, drop=True)
    # concat
    sample = pd.concat([pass_df, shoot_df, other_df], axis=1, sort=False)
    sample = sample.fillna(0)
    sample['period'] = use_period
    sample['last_team'] = last_team
    sample['last_event_type'] = last_event_type
    sample['last_x'] = last_x
    sample['last_y'] = last_y
    sample['last_event'] = last_event
    sample['last10_list'] = ','.join(last10_list)
    sample['team_id'] = sample.index
    sample['start'] = start
    sample['end'] = end
    # add new features
    data = sample
    def get_event_use(x, i):
        return '-'.join(x.split(',')[-i].split('-')[:2])
    data['last1_event'] = data.last10_list.map(lambda x: get_event_use(x,1))
    data['last2_event'] = data.last10_list.map(lambda x: get_event_use(x,2))
    data['pass1'] = data['pass']*data['pass1_rate']
    data['front1'] = data['front']*data['front1_rate']
    data['defend'] = data['tackle']+data['interception']
    data['takeon1'] = data['takeon1_rate']+data['takeon']
    data['positionC1'] = data['positionC1_rate']*data['positionC']
    data['positionL1'] = data['positionL1_rate']*data['positionL']
    data['positionR1'] = data['positionR1_rate']*data['positionR']
    data['positionB1'] = data['positionB1_rate']*data['positionB']
    data['last_position'] = data.last_event.map(lambda x:x.split('-')[2])
    data['position9'] = data.last10_list.map(lambda x:x.split(',')[9].split('-')[2])
    data['position8'] = data.last10_list.map(lambda x:x.split(',')[8].split('-')[2])
    data['position7'] = data.last10_list.map(lambda x:x.split(',')[7].split('-')[2])
    data['position6'] = data.last10_list.map(lambda x:x.split(',')[6].split('-')[2])
    data['position5'] = data.last10_list.map(lambda x:x.split(',')[5].split('-')[2])
    return data
  

In [16]:
# read game
xml = etree.parse(GAME_FILE)
game = xml.xpath('Game')[0]
# xml to dataframe
game_df = pd.concat([parse_event(x, event_dict, qualifier_dict, associate_dict) for x in game], axis=0)
game_df = game_df.fillna(value=UNK)
# cleaning
game_df.loc[game_df.event_type=='Deleted event', 
            ['length', 'direction', 'position', 'qualifier']] = UNK
game_df.loc[game_df.event_type=='Clearance', 
            ['direction', 'position']] = UNK
game_df['time'] = game_df['min'].astype('int')*60 + game_df['sec'].astype('int')
game_df = game_df.reset_index(drop=True)

use_df = game_df
start = game_df.time.iloc[0]
end = game_df.time.iloc[-1]


In [7]:
# ---player---
data = get_player_sample(game_df, start, end)
# feature transform
with open(player_transformer_file, 'rb') as handle:
    column_trans = pickle.load(handle)
X = column_trans.transform(data)
# predict
with open(player_model_file, 'rb') as handle:
    model = pickle.load(handle)
pred_player = model.predict(X)


/Users/logos/.pyenv/versions/3.5.6/lib/python3.5/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


In [8]:
pred_player

array([93289])

In [136]:
def concat_home_and_away_team(data):
    # concat home and away
    self_fields = {'pass1', 'front1', 'defend', 'ballTouch', 'shoot', 'foul', 'clearance', 'takeon1',
                  'positionC1', 'positionL1', 'positionR1', 'positionB1'}
    opp_fields = self_fields.copy()
    useful_fields = {'period', 'last_team', 'last_position', 'last1_event', 'last2_event', 'last_x', 'last_y', 
                     'last10_list', 'game'}
    home_df = data.loc[data.team_id=='1', list(self_fields.union(useful_fields))]
    home_df.reset_index(drop=True, inplace=True)
    away_df = data.loc[data.team_id=='0', list(opp_fields)]
    away_df.reset_index(drop=True, inplace=True)
    away_df.columns = away_df.columns + '_o'
    data_new = pd.concat([home_df, away_df], axis=1, sort=False)
    return data_new.reset_index(drop=True)

In [137]:
# ---team---
data = get_team_sample(game_df, start, end)
data = concat_home_and_away_team(data)

with open(next_team_transformer_file, 'rb') as handle:
    column_trans = pickle.load(handle)
X = column_trans.transform(data)

# predict
with open(next_team_model_file, 'rb') as handle:
    model = pickle.load(handle)
pred_next_team = model.predict(X)


/Users/logos/.pyenv/versions/3.5.6/lib/python3.5/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/logos/.pyenv/versions/3.5.6/lib/python3.5/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


In [140]:
def concat_home_and_away_xy(data):
    # concat home and away
    self_fields = {'pass1', 'front1', 'defend', 'ballTouch', 'shoot', 'foul', 'clearance', 'takeon1',
              'positionC1', 'positionL1', 'positionR1', 'positionB1',
              'positionC_rate', 'positionL_rate', 'positionR_rate', 'positionB_rate', 
               'short_rate', 'middle_rate', 'long_rate'}
    opp_fields = self_fields.copy()
    useful_fields = {'period', 'last_team', 'last_position', 'last1_event', 'last2_event', 'last10_list', 'game',
                    'last_x', 'last_y'}
#     if (predict_y):
#         useful_fields.add('last_y')
#     else:
#         useful_fields.add('last_x')
    add_pisition_fields = {'position9', 'position8', 'position7', 'position6', 'position5'}
    useful_fields = useful_fields.union(add_pisition_fields)
    home_df = data.loc[data.team_id=='1', list(self_fields.union(useful_fields))]
    home_df.reset_index(drop=True, inplace=True)
    away_df = data.loc[data.team_id=='0', list(opp_fields)]
    away_df.reset_index(drop=True, inplace=True)
    away_df.columns = away_df.columns + '_o'
    data_new = pd.concat([home_df, away_df], axis=1, sort=False)
    return data_new.reset_index(drop=True)

In [145]:
# ---xy---
data = get_team_sample(game_df, start, end)
data = concat_home_and_away_xy(data)

with open(next_x_transformer_file, 'rb') as handle:
    column_trans = pickle.load(handle)
X = column_trans.transform(data)

# predict
with open(next_x_model_file, 'rb') as handle:
    model = pickle.load(handle)
pred_next_x = model.predict(X)

with open(next_y_transformer_file, 'rb') as handle:
    column_trans = pickle.load(handle)
X = column_trans.transform(data)

# predict
with open(next_y_model_file, 'rb') as handle:
    model = pickle.load(handle)
pred_next_y = model.predict(X)

/Users/logos/.pyenv/versions/3.5.6/lib/python3.5/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/logos/.pyenv/versions/3.5.6/lib/python3.5/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
/Users/logos/.pyenv/versions/3.5.6/lib/python3.5/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


array([42.220707], dtype=float32)

In [153]:
result = [pred_player[0], pred_next_team[0], pred_next_y[0], pred_next_x[0]]
result = ','.join([str(x) for x in result])
with open('res_psgx.csv', 'w') as f:
    f.write(result)

In [154]:
with open('res_psgx.csv', 'w') as f:
    f.write(result)

In [155]:
import os

os.system()

TypeError: Required argument 'command' (pos 1) not found